In [1]:
import json
import os

In [2]:
# Load two files:
model_outputs = json.load(open('/Users/venkatakesavvenna/Sem6/RSAI/Mid_Project/dataset/nli_output/llama-70b_checker_final.json'))
human_outputs = json.load(open('/Users/venkatakesavvenna/Sem6/RSAI/Mid_Project/dataset/human_annotations/zero_context/nq_llama2_70b_chat_answers.json'))

In [3]:
# Print the keys 
print(model_outputs[0].keys())
print(human_outputs[0].keys())

dict_keys(['id', 'response', 'triplets', 'reference', 'Y', 'ys'])
dict_keys(['id', 'response', 'claude2_response_kg'])


In [4]:
# Make an array of the form [triplet, model_output, human_output]
final_array = []

# Check the ids, that are common in both the files
for model_output in model_outputs:
    for human_output in human_outputs:
        if model_output['id'] == human_output['id']:
            final_array.append([model_output["triplets"], model_output["ys"], [entry["human_label"] for entry in human_output["claude2_response_kg"]]])
            

In [5]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Categories
categories = ['Neutral', 'Entailment', 'Contradiction']

# Initialize confusion matrix
conf_matrix = np.zeros((len(categories), len(categories)))

# Count occurrences of each combination of model and human outputs
for entry in final_array:
    model_outputs = entry[1]
    human_outputs = entry[2]

    for model_output, human_output in zip(model_outputs, human_outputs):
        # Get indices of categories
        model_index = categories.index(model_output)
        human_index = categories.index(human_output)

        # Update confusion matrix
        conf_matrix[model_index][human_index] += 1

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Calculate precision, recall, and F1-score for each category
for i, category in enumerate(categories):
    true_positives = conf_matrix[i][i]
    false_positives = sum(conf_matrix[:, i]) - true_positives
    false_negatives = sum(conf_matrix[i, :]) - true_positives

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)

    print(f"\nCategory: {category}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1_score}")


Confusion Matrix:
[[565.  16.  39.]
 [155. 210.  25.]
 [ 50.  17.  66.]]

Category: Neutral
Precision: 0.7337662337662337
Recall: 0.9112903225806451
F1-score: 0.8129496402877697

Category: Entailment
Precision: 0.8641975308641975
Recall: 0.5384615384615384
F1-score: 0.6635071090047393

Category: Contradiction
Precision: 0.5076923076923077
Recall: 0.49624060150375937
F1-score: 0.5019011406844105
